In [1]:
import nest_asyncio
nest_asyncio.apply()

import os, json, time
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters


In [ ]:
BOT_TOKEN = "7556063014:AAGmI-zrPbqorpGaQAVt7yOvXiems3ciaY0"
JOB_DIR = "../jobs"

# Temporary buffer for awaiting profile links
user_pending_links = {}

async def handle_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text.strip()
    sender = update.effective_user.username or update.effective_user.id
    timestamp = int(time.time())

    parts = text.split(" ", 2)
    if len(parts) < 3:
        await update.message.reply_text("Format: /<command> <profile_user> <content>")
        return

    cmd_type = parts[0][1:].lower()
    profile_user = parts[1]
    content = parts[2]

    job = {
        "type": cmd_type,
        "user": profile_user,
        "content": content,
        "sender": sender,
        "timestamp": timestamp
    }

    # For commands needing profile links in follow-up message
    if cmd_type in ["connect", "tag", "comment_twitter", "comment_linkedin"]:
        user_pending_links[sender] = {
            "job": job,
            "expecting_links": True
        }
        await update.message.reply_text(f"Send the list of profile links for `{cmd_type}` (comma-separated).")
    else:
        write_job(job)
        await update.message.reply_text(f"✅ Job recorded for `{cmd_type}` as `{profile_user}`.")

async def handle_followup_links(update: Update, context: ContextTypes.DEFAULT_TYPE):
    sender = update.effective_user.username or update.effective_user.id
    text = update.message.text.strip()

    if sender not in user_pending_links:
        await update.message.reply_text("Unrecognized input. Use a command first.")
        return

    # Parse link input
    raw_links = [link.strip() for link in text.split(",") if link.strip()]
    job_data = user_pending_links[sender]["job"]
    job_data["profile_links"] = raw_links

    write_job(job_data)
    await update.message.reply_text(f"✅ Job + {len(raw_links)} links recorded for `{job_data['type']}`.")

    del user_pending_links[sender]

def write_job(job):
    fname = f"{job['type']}_{job['user']}_{job['timestamp']}.json"
    with open(os.path.join(JOB_DIR, fname), "w", encoding="utf-8") as f:
        json.dump(job, f, indent=2)

# App setup
app = ApplicationBuilder().token(BOT_TOKEN).build()
command_list = ["tweet", "linkedin", "connect", "tag", "comment_twitter", "comment_linkedin"]

app.add_handler(CommandHandler(command_list, handle_command))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_followup_links))

app.run_polling()
